In [14]:
from arcgis.geocoding import geocode
import pandas as pd 
from arcgis import GIS
import os, sys, time, csv
from arcgis import network
from arcgis.features import FeatureSet, FeatureLayer, Feature, FeatureLayerCollection
import time
from datetime import datetime

In [16]:
gis = GIS('https://arcgis.com', 'bhammersley_tech')

Enter password: ········


In [17]:
def Mygeocode(address,typo):
    m.clear_graphics()
    
    incident = geocode(address)[0]
    x = incident['location']['x']
    y = incident['location']['y']
    
    data =[[x,y,address,typo]]
    column = ['x','y','address', 'Type']
    df = pd.DataFrame(data,columns=column)
    
    sdf = pd.DataFrame.spatial.from_df(df)
    
    symbol = {
    "type" : "simple-marker",
    "style" : "diamond",
    "color" : [230,76,0,1]
    }
    popup = {
        "title" : incident['attributes']['PlaceName'],
        "content" : incident['attributes']['Place_addr']
        }
    m.draw(incident['location'], popup, symbol)
    
    return sdf

In [18]:
def closest_ambulance(incident_sdf, lyr):

    amb_fset=lyr.query(where="Availability='Yes'",out_sr=4326)

    analysis_url = gis.properties.helperServices.closestFacility.url
    cf_layer = network.ClosestFacilityLayer(analysis_url, gis=gis)
    
    incident_fset = incident_sdf.spatial.to_featureset()
    
    result = cf_layer.solve_closest_facility(incidents=incident_fset,
                                        facilities=amb_fset,
                                        default_target_facility_count=1,
                                        return_facilities=True,
                                        impedance_attribute_name='TravelTime',
                                        accumulate_attribute_names=['Kilometers','TravelTime'])
    
    
    dist = result['routes']['features'][0]['attributes']['Total_TravelTime']
    print('Incident is',dist,' minute drive from the nearest ambulance')
    
    
    return result

In [19]:
def closest_hospital(incident_sdf):
    
    hosp = gis.content.get('201b1f4ba8d04e20bb6c05d8912b6a27')
    hosp_lyr = hosp.layers[0]

    hosp_fset=hosp_lyr.query(out_sr=4326)
        
    analysis_url = gis.properties.helperServices.closestFacility.url
    cf_layer = network.ClosestFacilityLayer(analysis_url, gis=gis)
    
    incident_fset = incident_sdf.spatial.to_featureset()
    
    result = cf_layer.solve_closest_facility(incidents=incident_fset,
                                        facilities=hosp_fset,
                                        default_target_facility_count=1,
                                        return_facilities=True,
                                        impedance_attribute_name='TravelTime',
                                        accumulate_attribute_names=['Kilometers','TravelTime'])
    
    closest = result['routes']['features'][0]['attributes']['FacilityID']
    #print('Closest ambulance has ID:',closest)
    
    return result

In [20]:
def draw_route(result,colour):

    line_feat_list = []
    for line_dict in result['routes']['features']:
        f1 = Feature(line_dict['geometry'], line_dict['attributes'])
        line_feat_list.append(f1)
        
    routes_fset = FeatureSet(line_feat_list, 
                             geometry_type=result['routes']['geometryType'],
                             spatial_reference= result['routes']['spatialReference'])
    
    allocation_line_symbol_4 = {'type': 'esriSLS', 'style': 'esriSLSSolid',
                                'color': 
                                colour, 'width': 6}
    m.draw(routes_fset, symbol = allocation_line_symbol_4)

In [21]:
def change_att(closest_amb):
    
    amb = gis.content.get('4800f5a1dbbd4b1f8c7775aa6f4fa0f3')
    amb_lyr = amb.layers[0]
    amb_fset=amb_lyr.query(where="Availability='Yes'",out_sr=4326)
    
    closest_fac = closest_amb['routes']['features'][0]['attributes']['FacilityID']
    
    df = amb_fset.sdf
    
    OID = df.iloc[closest_fac-1]['OBJECTID']
    print('Ambulance with ID:',str(OID),'is closest to the incident')
    
    
    fset = amb.layers[0].query(where='OBJECTID='+str(OID))
    lyr = amb.layers[0]
    features = fset.features
    
    features[0].attributes['Availability'] = 'No'
    
    update = features[0]
    
    update_result = lyr.edit_features(updates=[update])
    
    return OID

In [22]:
def update_inc(result, sdf, OID, hosp_result):
    
    dist = result['routes']['features'][0]['attributes']['Total_TravelTime']
    
    x = sdf['SHAPE'][0]['x']
    y = sdf['SHAPE'][0]['y']
    sr = sdf['SHAPE'][0]['spatialReference']
    address = sdf['address'][0]
    Type = sdf['Type'][0]
    distance = float(str(dist)[:5])
    OID = OID
    dt = dTime[0:10]
    T = dTime[11:19]
    
    hosp = gis.content.get('201b1f4ba8d04e20bb6c05d8912b6a27')
    hosp_lyr = hosp.layers[0]
    hosp_sdf = hosp_lyr.query(out_sr=4326).sdf
    
    hosp_FID = hosp_result['routes']['features'][0]['attributes']['FacilityID']
    
    hosp_ID = hosp_sdf.iloc[hosp_FID-1]['OBJECTID']
    
    print('Hospital with ID:',str(hosp_ID),'is closest to the incident')
    
    update_dict = {"geometry":{"x":x, "y":y, "spatialReference":sr},"attributes":{"address":address,"Type":Type,"Time_to_ambulance":distance, "Ambulance_ID":OID,"Hospital_ID":hosp_ID, "Call_Date":dt,"Call_Time":T}}
    
    incidente = gis.content.get('aba37664307a4ba5ad4287fe839ea24d')
    inc_lyr = incidente.layers[0]
    inc_lyr.edit_features(adds=[update_dict])
    
    
    

In [25]:
def files_to_timestamp(path):
    files = [os.path.join(path, f) for f in os.listdir(path)]
    return dict ([(f, os.path.getmtime(f)) for f in files])

if __name__ == "__main__":

    path_to_watch = r"C:\Users\bhammersley\OneDrive - ESRI (UK) Ltd\Documents\Presentations\Berlin_2019\listen\watch_me"
    print( "Watching", path_to_watch)

    before = files_to_timestamp(path_to_watch)

    while 1:
        time.sleep (2)
        after = files_to_timestamp(path_to_watch)

        added = [f for f in after.keys() if not f in before.keys()]
        removed = [f for f in before.keys() if not f in after.keys()]
        modified = []

        for f in before.keys():
            if not f in removed:
                if os.path.getmtime(f) != before.get(f):
                    modified.append(f)

        if added: 
            print ("Added: ", ", ".join(added))
        if removed: 
            print( "Removed: ", ", ".join(removed))
        if modified: 
            m.add_layer(amb)
            
            dTime = str(datetime.now())[0:20]
            
            print('New emergency call at: ',dTime)
            data = list(csv.reader(open(r'C:\Users\bhammersley\OneDrive - ESRI (UK) Ltd\Documents\Presentations\Berlin_2019\listen\watch_me\test.csv')))
        
            typo = data[-1][0]
            address = data[-1][1]
            sdf = Mygeocode(address,typo)
            
            amb = gis.content.get('4800f5a1dbbd4b1f8c7775aa6f4fa0f3')
            amb_lyr = amb.layers[0]
            
            # find closest ambulance
            y = closest_ambulance(sdf, amb_lyr)
            
            #findclosest_hospital
            a = closest_hospital(sdf)
            
            #draw route ambulance to incident
            z = draw_route(y,[0,0,255,100])
            
            
            time.sleep(2)
            #draw route to hospital
            j = draw_route(a, [0, 255, 36, 100])
            
            # change ambulance availability to No
            z = change_att(y)
            
            # Update incidents
            update_inc(y,sdf, z, a)
            
            
            
            
            

        before = after

Watching C:\Users\bhammersley\OneDrive - ESRI (UK) Ltd\Documents\Presentations\Berlin_2019\listen\watch_me
New emergency call at:  2019-11-06 09:26:58.
Incident is 1.0517015960643104  minute drive from the nearest ambulance
Ambulance with ID: 5 is closest to the incident
Hospital with ID: 3 is closest to the incident
New emergency call at:  2019-11-06 09:28:04.
Incident is 6.942815985317724  minute drive from the nearest ambulance
Ambulance with ID: 6 is closest to the incident
Hospital with ID: 5 is closest to the incident


KeyboardInterrupt: 

In [23]:
m = gis.map()
m.center = {'spatialReference': {'latestWkid': 3857, 'wkid': 102100},'x': 9456.60874216188,'y': 6842717.624355748}
m.zoom = 14
m

MapView(layout=Layout(height='400px', width='100%'), zoom=14.0)

In [24]:
amb = gis.content.get('4800f5a1dbbd4b1f8c7775aa6f4fa0f3')
hosp = gis.content.get('201b1f4ba8d04e20bb6c05d8912b6a27')

m.add_layer(amb)
m.add_layer(hosp)